In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from pandas.plotting import autocorrelation_plot
from plotting_functions import \
run_diagnostic_plots, plot_acf, plot_trace, plot_sensor_output, \
get_fname_list, plot_layer_posteriors, plot_fieldobs, plot_hist_diagnostic, make_chainstat_table, \
make_chains
import plotting_functions
import visvis as vv
import diagplots2
import pymc3
import matplotlib

import geovis_notebook_version

%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Initialize plots

In [ ]:
author = 'dk' #dk' or 'rs

rp_param_title_list = ['Rock density', 'Magnetic susceptibility']
rp_layer_title_list = ['Layer 0', 'Layer 1']

if author == 'dk': # user input - for david's results
    dir_parent = '/Volumes/david_hd/obsidian/output/experiments/'
    dir_output_diagnostics = 'output/plots-diagnostic'
    dir_output_sensors = 'output/plots-sensors'
    fname_samples = 'output0.npz'

    dir_year = '2018'
    dir_month = '11'
    dir_day = '15'
    exp_str = '{}_{}_{}'.format(dir_month, dir_day, dir_year)
    num = '01'

    param_key_list = [
        'layer0rockProperties',
        'layer1rockProperties'
    ]
    param_idx_list = [0, 1]

    plot_function_list = [
        plot_acf,
        plot_trace,
        plot_hist_diagnostic,
        make_chainstat_table
    ]

    plot_function_names = [
        'acf',
        'trace',
        'hist',
        'chainstat-table',
    ]
    extension_list = [
        'eps',
        'eps',
        'eps',
        'csv',
    ]
    # for sensor plot output
    sensor_name_list = ['grav', 'mag'] #'grav' or 'mag'

    data_names_list = [
        'magSensors','magReadings','gravSensors','gravReadings'
    ]
    xp = lambda exp_str, num: os.path.join(exp_str, num)

elif author == 'rs': # user input - for richard's results
    dir_parent = '/Volumes/david_hd/obsidian/output/experiments/'
    dir_output_diagnostics = 'output/plots-diagnostic'
    dir_output_sensors = 'output/plots-sensors'
    #fname_samples = 'gascoyne_v4.npz'

    exp_str = 'gascoyne_v5_run'
    fname_samples = 'gascoyne_v5-rs-run03-thin1000.npz'
    #fname_samples = 'output.npz'
    #num_list = ['01', '02', '03', '04', '05', '06']
    num = '03'

    param_key_list = [
        'layer0rockProperties',
        'layer1rockProperties'
    ]
    param_idx_list = [0, 1]

    plot_function_list = [
        plot_acf,
        plot_trace,
        plot_hist_diagnostic,
        make_chainstat_table
    ]

    plot_function_names = [
        'acf',
        'trace',
        'hist',
        'chainstat-table'
    ]
    extension_list = [
        'eps',
        'eps',
        'eps',
        'csv',
    ]
    # for sensor plot output
    sensor_name_list = ['grav', 'mag'] #'grav' or 'mag'

    data_names_list = [
        'magSensors','magReadings','gravSensors','gravReadings'
    ]
    xp = lambda exp_str, num: exp_str + num

# Voxel plots

In [ ]:
# put together file paths
dir_output_voxels = 'output/plots-voxels'
search_str = 'voxel'
dir_voxels = 'voxels'
num = '01'
#dir_voxels = ''
if author == 'dk':
    fpath_in = get_fname_list(os.path.join(dir_parent, exp_str, num, dir_voxels), search_str)
    dir_out = os.path.join(dir_parent, exp_str, num, dir_output_voxels)
elif author == 'rs':
    fpath_in = get_fname_list(os.path.join(dir_parent, exp_str + num), search_str)
    dir_out = os.path.join(dir_parent, exp_str + num, dir_output_voxels)
print(fpath_in)
print(dir_out)

In [ ]:
fname_save_template_str = 'posterior-layer-{:02}.{}'
view = plot_layer_posteriors(fpath_in, dir_out)

In [ ]:
depth = 15000
no_voxels = 32
depth_per_voxel = depth / no_voxels
target_depth = 4000
no_vox_at_target = np.ceil(target_depth / depth_per_voxel)
print(no_vox_at_target)
depth_at_target = np.round((no_vox_at_target * depth_per_voxel) / 1000, 1)

In [ ]:
# view = a specific experiment in the list
view_idx = 0
layer_idx = 0

layer_voxels = view.meanlayer(layer_idx)
layer_voxels_rounded = layer_voxels
upper_val = 0.90
lower_val = 0.10
layer_voxels_rounded[layer_voxels > upper_val] = 1
layer_voxels_rounded[layer_voxels < lower_val] = 0

#plot_voxels = layer_voxels_rounded

# z, y, x is order of dimensions
cut_off_voxels = layer_voxels[:no_vox_at_target,:,:]

plot_voxels = cut_off_voxels

app = vv.use()
vv.figure(1)
vv.settings.figureSize = (500, 500)
vv.settings.defaultRelativeFontSize = 2.0
vv.xlabel('Eastings (km)')
vv.ylabel('Northings (km)')
vv.zlabel('Depth (km)')
#a.axis._fontSize = 100
#vv.xTicks([0, 15, 30])
a = vv.gca()
print(a.GetView())
#a.loc = (0, 0, 100)
#a.camera.fov = 10
a.daspect = 1, 1, -1
#a.daspect = 1, 1, -0.5
#a.camera.elevation = -90
#a.camera.azimuth = 30
a.camera.roll = 10
#a.camera.
a.axis.xTicks = {0:0, 16:7.5, 32:15}
a.axis.yTicks = {0:0, 16:7.5, 32:15}
a.axis.zTicks = {0:0, 8.2:depth_at_target}

#a.axis._tickFontSize = 20
#a.axis._ = 20

#vv._xticks = [0, 15, 30]
#print(a.GetView())
#print(a.camera)
#a.camera.
#a.SetView()
#t = vv.volshow(l, cm=vv.CM_JET)
#t = vv.volshow(layer_voxels, cm=vv.CM_JET, renderStyle='ray')
t = vv.volshow(plot_voxels, cm=vv.CM_JET)
#t = vv.volshow(layer_voxels, cm=vv.CM_JET[::-1])
vv.ColormapEditor(a)

vv.azimuth = 90
vv.elevation = -50

#a.camera.Reset()
#a.camera.elevation = 20

#a.Draw()
#a.SetView()

app.Run()

In [ ]:
# z, y, x is order of dimensions
layer_voxels0 = view.meanlayer(0)[:no_vox_at_target,:,:]
layer_voxels1 = view.meanlayer(1)[:no_vox_at_target,:,:]

vv.figure(1)
vv.settings.figureSize = (1000, 1000)
vv.settings.defaultRelativeFontSize = 2.0
vv.xlabel('Eastings (km)')
vv.ylabel('Northings (km)')
vv.zlabel('Depth (km)')
a = vv.gca()
a.daspect = 1, 1, -1
a.camera.roll = 10
a.axis.xTicks = {0:0, 16:7.5, 32:15}
a.axis.yTicks = {0:0, 16:7.5, 32:15}
a.axis.zTicks = {0:0, 8.2:depth_at_target}

#t = vv.volshow(layer_voxels0, cm=vv.CM_JET)
t = vv.volshow(layer_voxels1, cm=vv.CM_JET)
vv.ColormapEditor(a)
app = vv.use()

# good values for elevation: 30, 35, 40, 45
a.camera.elevation = 40
# good values for azimuth: -45
a.camera.azimuth = -30
a.camera.SetView()
#a.SetView()

app.Run()


# Plots of each side of the voxel cube

In [ ]:
"""plot_data = 1 - layer_voxels[0, :, :]
plt.imshow(plot_data)
plt.colorbar()"""

In [ ]:
try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass
plot_width = 10
plot_height = 10
fontsize = 20
tick_fontsize = 29
x_label = 'Eastings (km)'
y_label = 'Northings (km)'

In [ ]:
plot_data = np.flipud(layer_voxels[0, :, :])

f = plt.figure(figsize = (plot_width, plot_height))
ax = plt.gca()
ax.patch.set_facecolor('None')
out = plt.contourf(plot_data, origin = 'upper')
plt.colorbar()

In [ ]:
data_dict = {
    'North': layer_voxels[:,31,:],
    'South': layer_voxels[:,0,:],
    'East': layer_voxels[:,:,0],
    'West': layer_voxels[:,:,31],
}

In [ ]:
plot_width = 10
plot_height = 10
fontsize = 20
tick_fontsize = 29
x_label = 'Eastings (km)'
y_label = 'Northings (km)'

dist = 15000
no_voxels = 32
offset = 0.5
dist_per_voxel = dist / (no_voxels)
ticks_voxels = [0.5, 15.5, 31.5]
ticks_km = [t * dist_per_voxel for t in ticks_voxels]
tick_labels = [0, 7.5, 15]

fig, axes = plt.subplots(2,2,figsize=(10,10))
#plt.xlabel(x_label, fontsize = tick_fontsize)
#plt.ylabel(y_label, fontsize = tick_fontsize)
for ax,key in zip(axes.flatten(), data_dict):
    data = data_dict[key]
    ax.patch.set_facecolor('None')
    ax.set_xticks(ticks_voxels)
    ax.set_yticks(ticks_voxels)
    ax.set_xticklabels(tick_labels, fontsize=tick_fontsize - 10)
    ax.set_yticklabels(tick_labels, fontsize=tick_fontsize - 10)
    out = ax.contourf(data, origin = 'upper')
    ax.set_title(key, fontsize=tick_fontsize)
    cbar = plt.colorbar(out, ticks=cbar_ticks, ax=ax)
    cbar_ticks = [0, 1.05/2, 1.05]
    cbar.ax.tick_params(labelsize=tick_fontsize - 10)
    cbar_ticklabels = ['0\\%', '50\\%', '100\\%']
    cbar.set_ticks(cbar_ticks)
    cbar.set_ticklabels(cbar_ticklabels)
    ax.set_aspect('equal')
fig.text(0.03, 0.5, 'Northings', va='center', rotation='vertical', fontsize = tick_fontsize)
fig.text(0.4, 0.06, 'Eastings', va='center', fontsize = tick_fontsize)
plt.savefig('/Users/davidkohn/Desktop/voxel_sides.eps')